# Магические методы классов

Сначала рассмотрим пример, использующий магические методы

In [1]:
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
        
    def get_email_data(self):
        return {'name': self.name, 'email': self.email}
    
jane = User('Jane Doe', 'janedoe@example.com')
print(jane.get_email_data())

{'name': 'Jane Doe', 'email': 'janedoe@example.com'}


В целом, с магическими методами все просто - их надо знать=), а детальное разъяснение здесь бессмысленно, т.к. оно неполное. В лекции упоминались методы `__init__()`, `__new__()`, `__del__()`, `__str__()`, `__eq__()`, `__hash__()`, `__getattribute__()` (вызывается при всяком обращении к атрибуту), `__getattr__()` (вызывается, когда атрибут не найден), `__setattr__()` ,`__delattr__()` ,`__call__()` (определяет поведение при вызове класса), `__add__()`, `__getitem__()`, `__setitem__()`, ``, ``, ``

Интересный пример на декораторы и метод `__call__()`:

In [2]:
class Logger:
    def __init__(self, filename):
        self.filename = filename
        
    def __call__(self, func):
        def wrapped(*args, **kwargs):    
            with open(self.filename, 'a') as f:
                f.write('Oh, Danny boy...')
            return func(*args, **kwargs)
        return wrapped
    
logger = Logger('log.txt')

@logger
def completely_useless_function():
    pass

# Итераторы

Сначала сформулируем, чем итераторы отличаются от генераторов. Итератор - это любой объект, по которому можно проитерироваться, а генератор - это функция/метод, чьи значения возвращаются посредством конструкции `yield`. То есть, итератор - это более общая концепция, а генератор - более частная. Любой генератор является итератором, но не наоборот.

Для того, чтобы объект считался итератором, в нем должны быть реализованы либо методы `__iter__()` (возвращает объект для итерации в исходном состоянии) и `__next__()` (возвращает следующий элемент коллекции), либо метод `__getitem__()`. Реализуем свой вариант итератора:

In [3]:
class SquareIterator:
    def __init__(self, start, stop):
        self.current = start
        self.stop = stop
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current >= self.stop:
            raise StopIteration
        result = self.current ** 2
        self.current += 1
        return result
    
for num in SquareIterator(1, 4):
    print(num)

1
4
9


Код выше подразумевает, что итерируемся мы по самому объекту класса

In [4]:
class IndexIterable:
    def __init__(self, obj):
        self.obj = obj
        
    def __getitem__(self, index):
        return self.obj[index]
    
for char in IndexIterable('abcde'):
    print(char)

a
b
c
d
e


# Контекстные менеджеры

Мы с контекстными менеджерами регулярно работаем при работе с файлами через конструкцию `with`:

```python
with open('access_log.log', 'a') as f:
    f.write('New access')
```

Попробуем определить собственный контекстный менеджер:

In [5]:
class open_for_append:
    def __init__(self, filename):
        self.f = open(filename, 'a')
        
    def __enter__(self):
        return self.f
    
    def __exit__(self, *args):
        self.f.close()

Любопытная особенность контекстных менеджеров состоит в том, что там можно обрабатывать исключения:

In [6]:
class suppress_exception:
    def __init__(self, exc_type):
        self.exc_type = exc_type
        
    def __enter__(self):
        return
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type == self.exc_type:
            print("It's okay")
            return True

with suppress_exception(ZeroDivisionError):
    infinity = 1/0

It's okay


Теперь напишем свой контекстный менеджер, который считает время, которое код провел внутри него.

In [7]:
from datetime import datetime
import time

class timer:
    def __init__(self):
        self.enter_time = datetime.now()
        
    def current_time(self):
        return datetime.now() - self.enter_time
        
    def __enter__(self):
        return self
        
    def __exit__(self, *args):
        print(self.current_time())
        
with timer() as t:
    time.sleep(1)
    print(t.current_time())
    time.sleep(1)

0:00:01.002828
0:00:02.017125


# Задание 1

В этом задании вам нужно создать интерфейс для работы с файлами. Интерфейс должен предоставлять следующие возможности по работе с файлами:

- чтение из файла, метод __read__ возвращает строку с текущим содержанием файла  

- запись в файл, метод __write__ принимает в качестве аргумента строку с новым содержанием файла

- сложение объектов типа __File__, результатом сложения является объект класса __File__, при этом создается новый файл и файловый объект, в котором содержимое второго файла добавляется к содержимому первого файла. Новый файл должен создаваться в директории, полученной с помощью функции `tempfile.gettempdir`. Для получения нового пути можно использовать `os.path.join`.

- возвращать в качестве строкового представления объекта класса __File__ полный путь до файла

- поддерживать протокол итерации, причем итерация проходит по строкам файла

При создании экземпляра класса __File__ в конструктор передается полный путь до файла на файловой системе. Если файла с таким путем не существует, он должен быть создан при инициализации.

__Пример работы:__

```python
>>> import os.path
>>> from solution import File
>>> path_to_file = 'some_filename'
>>> os.path.exists(path_to_file)
False
>>> file_obj = File(path_to_file)
>>> os.path.exists(path_to_file)
True
>>> print(file_obj)
some_filename
>>> file_obj.read()
''
>>> file_obj.write('some text')
9
>>> file_obj.read()
'some text'
>>> file_obj.write('other text')
10
>>> file_obj.read()
'other text'
>>> file_obj_1 = File(path_to_file + '_1')
>>> file_obj_2 = File(path_to_file + '_2')
>>> file_obj_1.write('line 1\n')
7
>>> file_obj_2.write('line 2\n')
7
>>> new_file_obj = file_obj_1 + file_obj_2
>>> isinstance(new_file_obj, File)
True
>>> print(new_file_obj)
C:\Users\Media\AppData\Local\Temp\71b9e7b695f64d85a7488f07f2bc051c
>>> for line in new_file_obj:
....    print(ascii(line))  
'line 1\n'
'line 2\n'
>>> new_path_to_file = str(new_file_obj)
>>> os.path.exists(new_path_to_file)
True
>>> file_obj_3 = File(new_path_to_file)
>>> print(file_obj_3)
C:\Users\Media\AppData\Local\Temp\71b9e7b695f64d85a7488f07f2bc051c
>>>
```


In [8]:
import os.path
import tempfile

class File:
    def __init__(self, file_name):
        self.file_name = file_name
        self.data = None
        if not os.path.exists(self.file_name):
            open(self.file_name, 'a').close()
        
    def read(self):
        with open(self.file_name) as fd:
            return fd.read()
        
    def write(self, data):
        with open(self.file_name, 'w') as fd:
            fd.write(data)
            
    def __str__(self):
        return self.file_name
    
    def __add__(self, other):
        file_path = ''
        with tempfile.NamedTemporaryFile(dir=tempfile.gettempdir(), mode='w', delete=False) as fd:
            fd.write(self.read() + '\n' + other.read(), )
            file_path = os.path.join(tempfile.gettempdir(), fd.name)
            #fd.flush()
        return File(file_path)
    
    def __getitem__(self, index):
        if self.data is None:
            with open(self.file_name) as fd:
                self.data = fd.readlines()
        return self.data[index]

In [9]:
f1 = File('tmp.data')
f1.read()
f1.write('lorum ipsum...')
f1.read()
f2 = File('foo.data')
f2.write('si vis pacem...')
f3 = f1 + f2
print(f3)

C:\Users\kb255048\AppData\Local\Temp\tmpbmx5jxn5


In [10]:
print(f3.read())

lorum ipsum...
si vis pacem...


In [11]:
for line in f3:
    print(ascii(line))

'lorum ipsum...\n'
'si vis pacem...'


# Дескрипторы

Мы их уже использовали ранее при работе со свойствами, т.е. `property()` - это дескриптор. Говоря более формально, дескриптор в Python - это средства языка, позволяющее переопределить работу кода, отвечающего за обращение к какому-либо атрибуту объекта. В качестве примера определим свой собственный дескриптор:

In [12]:
class Descriptor:
    def __get__(self, obj, obj_type):
        print('get')
    
    def __set__(self, obj, value):
        print('set')
    
    def __delete__(self, obj):
        print('delete')
        
class Class:
    attr = Descriptor()
    
instance = Class()

В коде выше мы фактически переопределили поведение класса `Class` при доступе к атрибуту `attr`, т.е. при его чтении, записи или удалении. 

Касаемо дескрипторов важно понимать их приоритет над работой с атрибутами:  
1. Если в дескрипторе переопределены методы `__get__()` и `__set__()`, то он называется data descriptor и имеет приоритет как при чтении, так и при записи в атрибут.  
2. Если в дескрипторе переопределен только метод `__get__()`, то он называется non-data descriptor и имеет приоритет только при чтении, но не при записи. В таком случае при записи данные пойдут напрямую в атрибут.

Также, не совсем очевидно предназначение параметров `obj` и `obj_type`. Дело в том, что дескриптор можно вызывать не только через объект, но и через класс, и тогда, очевидно, поведение должно быть разным. Параметры `obj` и `obj_type` позволяют определить, из какого объекта какого класса вызван метод.

In [13]:
instance.attr

get


In [14]:
instance.attr = 42

set


In [15]:
del instance.attr

delete


В коде выше мы фактически переопределили поведение класса `Class` при доступе к атрибуту `attr`, т.е. при его чтении, записи или удалении. 

Касаемо дескрипторов важно понимать их приоритет над работой с атрибутами:  
1. Если в дескрипторе переопределены методы `__get__()` и `__set__()`, то он называется data descriptor и имеет приоритет как при чтении, так и при записи в атрибут.  
2. Если в дескрипторе переопределен только метод `__get__()`, то он называется non-data descriptor и имеет приоритет только при чтении, но не при записи. В таком случае при записи данные пойдут напрямую в атрибут.

Напишем более реальный пример дескриптора, который бы логировал каждое изменение атрибута класса.

В коде выше мы фактически переопределили поведение класса `Class` при доступе к атрибуту `attr`, т.е. при его чтении, записи или удалении. 

Касаемо дескрипторов важно понимать их приоритет над работой с атрибутами:  
1. Если в дескрипторе переопределены методы `__get__()` и `__set__()`, то он называется data descriptor и имеет приоритет как при чтении, так и при записи в атрибут.  
2. Если в дескрипторе переопределен только метод `__get__()`, то он называется non-data descriptor и имеет приоритет только при чтении, но не при записи. В таком случае при записи данные пойдут напрямую в атрибут.

In [16]:
class ImportantValue:
    def __init__(self, amount):
        self.amount = amount
        
    def __get__(self, obj, obj_type):
        return self.amount
    
    def __set__(self, obj, value):
        with open('acc_log.txt', 'a') as fd:
            fd.write(str(value) + '\n')
        self.amount = value
        
class Account:
    amount = ImportantValue(100)
    
bobs_account = Account()
bobs_account.amount = 150
bobs_account.amount = 200
bobs_account.amount = 42

with open('acc_log.txt', 'r') as fd:
    print(fd.readlines())

['150\n', '200\n', '42\n', '150\n', '200\n', '42\n', '150\n', '200\n', '42\n', '150\n', '200\n', '42\n']


Ниже приведен код, использующий декоратор `property`. Раньше мы уже работали с декораторами `property.getter` и `property.setter`. Но то, что приведено ниже позволяет функцию использовать как атрибут без возможности записи.

In [17]:
class User:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'

Попробуем сделать свой аналог такого `property`:

In [18]:
class Property:
    def __init__(self, getter):
        self.getter = getter
        
    def __get__(self, obj, obj_type=None):
        if obj is None:
            return self
        return self.getter(obj)

Здесь наиболее интересным является метод `__get__()`. Он на входе кроме прочего принимает объект, из которого данный метод вызывается и тип этого объекта. При этом код предусматривает, что метод может быть вызван из класса, и тогда ничего не делает. А теперь применим этот `Property`:

In [19]:
class Class:
    @property
    def original(self):
        return 'original'
    
    @Property
    def custom_sugar(self):
        return 'custom sugar'
    
    def custom_pure(self):
        return 'custom pure'
    
    custom_pure = Property(custom_pure)
    
obj = Class()
print(obj.original)
print(obj.custom_sugar)
print(obj.custom_pure)

original
custom sugar
custom pure


С помощью дескрипторов в Python реализовано достаточно много функционала. Например, с помощью дескрипторов мы можем явно ограничить список атрибутов объекта. Для этого пригодится конструкция `__slots__`, с помощью которой Python определяет дескрипторы для каждого из атрибутов. При этом в объекте не ведется словарь атрибутов.

In [20]:
class Class:
    __slots__ = ['anakin']
    
    def init(self):
        self.anakin = 'the choosen one'
        
obj = Class()

try:
    obj.luke = 'the choosen too'
except AttributeError as e:
    print(e)

'Class' object has no attribute 'luke'


# Метаклассы

Метакласс - это класс, который может создавать другие классы. Важно понимать, что это не класс-родитель, а именно класс-создатель. Пример:

In [21]:
class Class:
    pass
obj = Class()
print(type(obj))
print(type(Class))
print(issubclass(Class, type))
print(issubclass(Class, object))
print(type(type))

<class '__main__.Class'>
<class 'type'>
False
True
<class 'type'>


Важно понимать, что в Python конструкция

```python
class Class:
    pass
```

эквивалентна следующему вызову:

```python
NewClass = type('NewClass', (), {})
```

Из сказанного выше следует, что `type` - это метакласс. Создадим свой собственный метакласс, который сможет создавать объекты:

In [22]:
class Meta(type):
    def __new__(cls, name, parents, attrs):
        print(f'Creating {name}...')
        
        if 'class_id' not in attrs:
            attrs['class_id'] = name.lower()
            
        return super().__new__(cls, name, parents, attrs)

class A(metaclass=Meta):
    pass

Creating A...


На практике метаклассы оказались полезны при поиске подклассов текущего класса. Попробуем реализовать такой функционал самостоятельно:

In [23]:
class Meta(type):
    def __init__(cls, name, bases, attrs):
        print(f'Initializing {name}')
        if not hasattr(cls, 'registry'):
            cls.registry = {}
        else:
            cls.registry[name.lower()] = cls
        super().__init__(name, bases, attrs)
        
class base(metaclass=Meta): pass
class a(base): pass
class b(base): pass

print(base.registry)
print(base.__subclasses__())

Initializing base
Initializing a
Initializing b
{'a': <class '__main__.a'>, 'b': <class '__main__.b'>}
[<class '__main__.a'>, <class '__main__.b'>]


# Абстрактные методы

По-красивому класс с абстрактными методами можно определить вот так:

In [24]:
from abc import ABCMeta, abstractmethod

class Sender(metaclass=ABCMeta):
    @abstractmethod
    def send(self):
        "Do something..."

class Child(Sender): pass

try:
    Child()
except TypeError as err:
    print(err)

Can't instantiate abstract class Child with abstract method send


На практике же абстрактные методы реализуются куда проще:

In [25]:
class PythonWay:
    def send(self):
        raise NotImplementedError

# Задание 2

Часто при зачислении каких-то средств на счет с нас берут комиссию. Давайте реализуем похожий механизм с помощью дескрипторов. Напишите дескриптор Value, который будет использоваться в нашем классе Account.

In [26]:
class Value:
    def __init__(self):
        self.amount = 0
        
    def __get__(self, obj, obj_type=None):
        if obj is None:
            return self
        return self.amount
    
    def __set__(self, obj, value):
        self.amount = self.amount + value * (1 - obj.commission)

class Account:
    amount = Value()
    
    def __init__(self, commission):
        self.commission = commission
        
new_account = Account(0.1)
new_account.amount = 100
print(new_account.amount)

90.0
